In [1]:
'''
    local_exec
    IBM/Dell Qiskit Runtime Emulator Demo
    July 26, 2021
    
    
    The following notebook walks through a (simple) example usage of the 
    Qiskit Runtime Emulator in a local execution environment: i.e. potentially
    using a locally installed simulator or a remote call directly from a
    local machine to a remote simulator or QPU.
'''

from qiskit_emulator import EmulatorProvider
from qiskit import QuantumCircuit
import logging
import requests
import time
import os

'''
    If the program that interacts with the simulator/QPU is small enough,
    it can be stored as a string in the file that interfaces with the 
    provider. Both directories and files can be taken as input, as well.

'''

RUNTIME_PROGRAM = """
# This code is part of qiskit-runtime.
#
# (C) Copyright IBM 2021.
#
# This code is licensed under the Apache License, Version 2.0. You may
# obtain a copy of this license in the LICENSE.txt file in the root directory
# of this source tree or at http://www.apache.org/licenses/LICENSE-2.0.
#
# Any modifications or derivative works of this code must retain this
# copyright notice, and modified files need to carry a notice indicating
# that they have been altered from the originals.
from qiskit.compiler import transpile, schedule


def main(
    backend,
    user_messenger,
    circuits,
    **kwargs,
):
    circuits = transpile(
        circuits,
    )
    
    user_messenger.publish({'results': 'intermittently'}, final=False)

    if not isinstance(circuits, list):
        circuits = [circuits]

    # Compute raw results using either simulator or QPU backend.
    result = backend.run(circuits, **kwargs).result()

    user_messenger.publish(result.to_dict(), final=True)
"""

RUNTIME_PROGRAM_METADATA = {
    "max_execution_time": 600,
    "description": "Qiskit test program"
}

PROGRAM_PREFIX = 'qiskit-test'
REMOTE_RUNTIME = os.getenv("SERVER_URL") 

logging.basicConfig(level=logging.DEBUG)

In [2]:
'''
    The EmulatorProvider is an interface that offers a choice of runtime
    (local or remote) and lists which backends are available locally.
    
    Both the client process and the job process interact with the 
    EmulatorProvider - the client mainly uses it for the runtime while the 
    job process selects a backend from the list.
'''

provider = EmulatorProvider()

INFO:qiskit.compiler.assembler:Total Assembly Time - 0.08082 (ms)
INFO:qiskit.compiler.assembler:Total Assembly Time - 0.06008 (ms)
INFO:qiskit.compiler.assembler:Total Assembly Time - 0.05531 (ms)
INFO:qiskit.compiler.assembler:Total Assembly Time - 0.05484 (ms)
INFO:qiskit.compiler.assembler:Total Assembly Time - 0.05317 (ms)
INFO:qiskit.compiler.assembler:Total Assembly Time - 0.06318 (ms)
INFO:qiskit.compiler.assembler:Total Assembly Time - 0.05341 (ms)
INFO:qiskit.compiler.assembler:Total Assembly Time - 0.05341 (ms)
INFO:qiskit.compiler.assembler:Total Assembly Time - 0.06127 (ms)
INFO:qiskit.compiler.assembler:Total Assembly Time - 0.05293 (ms)
INFO:qiskit.compiler.assembler:Total Assembly Time - 0.05651 (ms)


In [3]:
'''
    The runtime is a service provided that allows clients to upload, update,
    view, and run programs inside an execution environment. Since the client
    has not specified a remote runtime to the provider it defaults to local.
'''

program_id = provider.runtime.upload_program(RUNTIME_PROGRAM, metadata=RUNTIME_PROGRAM_METADATA)
print(f"PROGRAM ID: {program_id}")

DEBUG:qiskit_emulator.emulator_runtime_service:Have string: 
# This code is part of qiskit-runtime.
#
# (C) Copyright IBM 2021.
#
# This code is licensed under the Apache License, Version 2.0. You may
# obtain a copy of this license in the LICENSE.txt file in the root directory
# of this source tree or at http://www.apache.org/licenses/LICENSE-2.0.
#
# Any modifications or derivative works of this code must retain this
# copyright notice, and modified files need to carry a notice indicating
# that they have been altered from the originals.
from qiskit.compiler import transpile, schedule


def main(
    backend,
    user_messenger,
    circuits,
    **kwargs,
):
    circuits = transpile(
        circuits,
    )
    
    user_messenger.publish({'results': 'intermittently'}, final=False)

    if not isinstance(circuits, list):
        circuits = [circuits]

    # Compute raw results using either simulator or QPU backend.
    result = backend.run(circuits, **kwargs).result()

    user_mess

PROGRAM ID: 4ed007ab4b082f2b


In [4]:
programs = provider.runtime.pprint_programs(refresh=True)

4ed007ab4b082f2b:
  Name: 4ed007ab4b082f2b
  Description: Qiskit test program
  Version: 0
  Creation date: 04/08/2021 12:54:06
  Max execution time: 600
  Input parameters:
    none
  Interim results:
    none
  Returns:
    none


In [5]:
provider.runtime.update_program(program_id, description="IBM/Dell Updated Qiskit Runtime Program")

True

In [6]:
programs = provider.runtime.pprint_programs(refresh=True)

4ed007ab4b082f2b:
  Name: 4ed007ab4b082f2b
  Description: IBM/Dell Updated Qiskit Runtime Program
  Version: 0
  Creation date: 04/08/2021 12:54:06
  Max execution time: 600
  Input parameters:
    none
  Interim results:
    none
  Returns:
    none


In [7]:
'''
    Setting up QuantumCircuit objects from Qiskit to provide as inputs
    to the program we uploaded.
'''

qc = QuantumCircuit(2, 2)
qc.h(0)
qc.cx(0, 1)
qc.measure([0, 1], [0, 1])

program_inputs = {
    'circuits': qc,
}

In [8]:
'''
    Through the provider we can run an instance of our program with the
    inputs we have just created. 
    
    When we run a job locally, we launch a separate process and communicate with the
    client process via a socket connection.
'''

job = provider.runtime.run(program_id, options=None, inputs=program_inputs)

DEBUG:qiskit_emulator.emulator_runtime_job:41177
DEBUG:root:starting to listen to port 41177
DEBUG:qiskit_emulator.emulation_executor:creating temp directory at /tmp/tmpmj7cxxsj
DEBUG:qiskit_emulator.emulation_executor:finished writing to /tmp/tmpmj7cxxsj/program.py
DEBUG:qiskit_emulator.emulation_executor:finished writing to /tmp/tmpmj7cxxsj/params.json
DEBUG:qiskit_emulator.emulation_executor:finished writing to /tmp/tmpmj7cxxsj/executor.py
DEBUG:qiskit_emulator.emulation_executor:starting ['/home/geoff/miniconda3/envs/qruntime/bin/python3', '/tmp/tmpmj7cxxsj/executor.py']
DEBUG:root:accepted client connection from ('127.0.0.1', 35480)
DEBUG:qiskit_emulator.emulator_runtime_job:result: got final result.
DEBUG:qiskit_emulator.emulator_runtime_job:local thread: exiting
DEBUG:qiskit_emulator.emulation_executor:finished executing ['/home/geoff/miniconda3/envs/qruntime/bin/python3', '/tmp/tmpmj7cxxsj/executor.py']
DEBUG:qiskit_emulator.emulation_executor:stdout: using default backend: aer

In [9]:
'''
    We can obtain job results and specify a timeout if desired.
    If we do not specify a timeout, the function call returns immediately,
    either with None or the final results if they are present.
'''

results = job.result(timeout=60)
print(results)

{'backend_name': 'aer_simulator', 'backend_version': '0.8.2', 'qobj_id': 'd1d4b47c-f50a-4601-94d1-dcdeb9df36bb', 'job_id': '3b1c0696-de06-4a1a-bed2-0e18d17a6537', 'success': True, 'results': [{'shots': 1024, 'success': True, 'data': {'counts': {'0x0': 526, '0x3': 498}}, 'meas_level': 2, 'header': {'clbit_labels': [['c', 0], ['c', 1]], 'creg_sizes': [['c', 2]], 'global_phase': 0.0, 'memory_slots': 2, 'metadata': None, 'n_qubits': 2, 'name': 'circuit-2', 'qreg_sizes': [['q', 2]], 'qubit_labels': [['q', 0], ['q', 1]]}, 'status': 'DONE', 'seed_simulator': 4284887258, 'metadata': {'parallel_state_update': 2, 'measure_sampling': True, 'method': 'stabilizer', 'parallel_shots': 1, 'device': 'CPU', 'fusion': {'enabled': False}}, 'time_taken': 0.0022063390000000003}], 'date': '2021-08-04T12:54:22.126370', 'status': 'COMPLETED', 'header': {'backend_name': 'aer_simulator', 'backend_version': '0.8.2'}, 'metadata': {'time_taken': 0.002378308, 'parallel_experiments': 1, 'omp_enabled': True, 'max_gpu_

In [10]:
'''
    We can also provide a callback function to the runtime.
    A thread launched in the client process that polls for messages
    calls the callback function whenever it receives a non-final message.
'''

def callback_function(msg):
    print(f'******************\n\n\nFrom Callback Function: {msg}\n\n\n******************')
    

In [11]:
job = provider.runtime.run(program_id, inputs=program_inputs, options=None, callback=callback_function)

DEBUG:qiskit_emulator.emulator_runtime_job:60719
DEBUG:root:starting to listen to port 60719
DEBUG:qiskit_emulator.emulation_executor:creating temp directory at /tmp/tmpwdpy31ri
DEBUG:qiskit_emulator.emulation_executor:finished writing to /tmp/tmpwdpy31ri/program.py
DEBUG:qiskit_emulator.emulation_executor:finished writing to /tmp/tmpwdpy31ri/params.json
DEBUG:qiskit_emulator.emulation_executor:finished writing to /tmp/tmpwdpy31ri/executor.py
DEBUG:qiskit_emulator.emulation_executor:starting ['/home/geoff/miniconda3/envs/qruntime/bin/python3', '/tmp/tmpwdpy31ri/executor.py']
DEBUG:root:accepted client connection from ('127.0.0.1', 59240)
DEBUG:qiskit_emulator.emulator_runtime_job:result: got final result.
DEBUG:qiskit_emulator.emulator_runtime_job:local thread: exiting
DEBUG:qiskit_emulator.emulation_executor:finished executing ['/home/geoff/miniconda3/envs/qruntime/bin/python3', '/tmp/tmpwdpy31ri/executor.py']
DEBUG:qiskit_emulator.emulation_executor:stdout: using default backend: aer

******************


From Callback Function: {'results': 'intermittently'}


******************


In [12]:
'''
    We can also specify a different backend to run the quantum portions on. 
    The default backend is the Aer simulator.
'''

program_inputs['backend_name'] = 'emulator'
job = provider.runtime.run(program_id, inputs=program_inputs, options=None, callback=callback_function)

DEBUG:qiskit_emulator.emulator_runtime_job:59417
DEBUG:root:starting to listen to port 59417
DEBUG:qiskit_emulator.emulation_executor:creating temp directory at /tmp/tmpx63qoxxz
DEBUG:qiskit_emulator.emulation_executor:finished writing to /tmp/tmpx63qoxxz/program.py
DEBUG:qiskit_emulator.emulation_executor:finished writing to /tmp/tmpx63qoxxz/params.json
DEBUG:qiskit_emulator.emulation_executor:finished writing to /tmp/tmpx63qoxxz/executor.py
DEBUG:qiskit_emulator.emulation_executor:starting ['/home/geoff/miniconda3/envs/qruntime/bin/python3', '/tmp/tmpx63qoxxz/executor.py']
DEBUG:root:accepted client connection from ('127.0.0.1', 49900)
DEBUG:qiskit_emulator.emulator_runtime_job:result: got final result.
DEBUG:qiskit_emulator.emulator_runtime_job:local thread: exiting
DEBUG:qiskit_emulator.emulation_executor:finished executing ['/home/geoff/miniconda3/envs/qruntime/bin/python3', '/tmp/tmpx63qoxxz/executor.py']
DEBUG:qiskit_emulator.emulation_executor:stdout: {'circuits': <qiskit.circu

******************


From Callback Function: {'results': 'intermittently'}


******************
